<h1> Text Classification using TensorFlow on Cloud ML Engine </h1>

This notebook illustrates:
<ol>
<li> Creating datasets for Machine Learning using BigQuery
<li> Creating a text classification model using the high-level Estimator API 
<li> Training on Cloud ML Engine
<li> Deploying model
<li> Predicting with model
</ol>

In [4]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [5]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [6]:
%datalab project set -p $PROJECT

In [ ]:
!pip install --upgrade tensorflow

In [2]:
import tensorflow as tf
print tf.__version__

1.2.0


The idea is to look at the title of a newspaper article and figure out whether the article came from the New York Times or from TechCrunch. There are very sophisticated approaches that we can try, but for now, let's go with something very simple.

<h2> Data exploration and preprocessing in BigQuery </h2>
<p>
What does the Hacker News dataset look like?

In [10]:
%bq query
SELECT
  url, title, score
FROM
  `bigquery-public-data.hacker_news.stories`
WHERE
  LENGTH(title) > 10
  AND score > 10
LIMIT 10

url,title,score
,Ask HN: What are some good gift ideas for hacker types?,44
,"Ask HN: After Google Apps and Outlook, which email provider for custom domain?",22
http://code.ipstenu.org/2011/the-legality-of-forking/,The Legality of Forking,17
https://medium.com/@polarrist/where-are-chernobyl-s-children-a-photojournalist-s-honest-project-in-the-age-of-disaster-tourism-4cd333ab80c7,Where Are Chernobyl’s Children?,50
http://www.bbc.co.uk/news/technology-19597437,Twitter hands over messages at heart of Occupy case,61
http://weblogs.asp.net/fbouma/archive/2013/08/13/windows-store-account-getting-rid-of-it-is-as-hard-as-signing-up.aspx,Windows Store dev account: getting rid of it is as hard as signing up,51
http://www.forbes.com/sites/andyellwood/2012/01/18/being-a-regular/,Being A Regular,28
,Ask HN: The Road to Becoming an Angel or VC,12
http://personalmba.com/best-business-books/,Best Business Books,20
http://www.couch.io/migrating-to-couchdb,Migrating to CouchDB,48


Let's do some regular expression parsing in BigQuery to get the source of the newspaper article from the URL. For example, if the url is http://mobile.nytimes.com/...., I want to be left with <i>nytimes</i>. To ensure that the parsing works for all URLs of interest, I'll group by the source to make sure there are no weird names left. This was an iterative process.

In [26]:
query="""
SELECT
  ARRAY_REVERSE(SPLIT(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.'))[OFFSET(1)] AS source,
  COUNT(title) AS num_articles
FROM
  `bigquery-public-data.hacker_news.stories`
WHERE
  REGEXP_CONTAINS(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.com$')
  AND LENGTH(title) > 10
GROUP BY
  source
ORDER BY num_articles DESC
LIMIT 10
"""

In [27]:
import google.datalab.bigquery as bq
df = bq.Query(query).execute().result().to_dataframe()
df

,source,num_articles
0,blogspot,41386
1,github,36525
2,techcrunch,30891
3,youtube,30848
4,nytimes,28787
5,medium,18422
6,google,18235
7,wordpress,17667
8,arstechnica,13749
9,wired,12841


Now that we have good parsing of the URL to get the source, let's put together a dataset of source and titles. This will be our labeled dataset for machine learning.

In [92]:
query="""
SELECT source, REGEXP_REPLACE(title, '[^a-zA-Z0-9 $.-]', ' ') AS title FROM
(SELECT
  ARRAY_REVERSE(SPLIT(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.'))[OFFSET(1)] AS source,
  title
FROM
  `bigquery-public-data.hacker_news.stories`
WHERE
  REGEXP_CONTAINS(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.com$')
  AND LENGTH(title) > 10
)
WHERE (source = 'github' OR source = 'nytimes' OR source = 'techcrunch')
"""
df = bq.Query(query + " LIMIT 10").execute().result().to_dataframe()
df.head()

,source,title
0,github,Opinionated Dress Color Simulator
1,techcrunch,Meteor Raises $20M
2,techcrunch,DataSift Raises $15M To Help Businesses Mine A...
3,github,Writing Cross-Platform Games in Rust Using Piston
4,github,DAws Advanced Web Shell


For ML training, we will need to split our dataset into training and evaluation datasets (and perhaps an independent test dataset if we are going to do model or feature selection based on the evaluation dataset).  A simple way to do this is to use the hash of a well-distributed column in our data (See https://www.oreilly.com/learning/repeatable-sampling-of-data-sets-in-bigquery-for-machine-learning).
<p>
So, let's do that and save the results as CSV files.

In [93]:
traindf = bq.Query(query + " AND MOD(ABS(FARM_FINGERPRINT(title)),4) > 0").execute().result().to_dataframe()
evaldf  = bq.Query(query + " AND MOD(ABS(FARM_FINGERPRINT(title)),4) = 0").execute().result().to_dataframe()
traindf.head()

,source,title
0,github,Awesome per directory history for ZSH
1,github,PHP class which implements the Elo rating system
2,github,Comic Sans Everything
3,github,Amazing Flat version of Twitter Bootstrap
4,github,A year of fun and hard work on learning Dart


In [86]:
traindf['source'].value_counts()

github        27445
techcrunch    23131
nytimes       21586
Name: source, dtype: int64

In [87]:
evaldf['source'].value_counts()

github        9080
techcrunch    7760
nytimes       7201
Name: source, dtype: int64

In [94]:
traindf.to_csv('train.csv', header=False, index=False, encoding='utf-8', sep='\t')
evaldf.to_csv('eval.csv', header=False, index=False, encoding='utf-8', sep='\t')

In [95]:
!head -3 train.csv

github	Awesome per directory history for ZSH
github	PHP class which implements the Elo rating system
github	Comic Sans Everything


In [96]:
!wc -l *.csv

  24041 eval.csv
  72162 train.csv
  72164 vocab.csv
 168367 total


In [97]:
%bash
gsutil cp *.csv gs://${BUCKET}/txtcls1/

Copying file://eval.csv [Content-Type=text/csv]...
Copying file://train.csv [Content-Type=text/csv]...
Copying file://vocab.csv [Content-Type=text/csv]...
\
Operation completed over 3 objects/9.4 MiB.                                      


<h2> TensorFlow code </h2>

Please explore the code in this <a href="txtcls1/trainer">directory</a> -- <a href="txtcls1/trainer/model.py">model.py</a> contains the key TensorFlow model and <a href="txtcls1/trainer/task.py">task.py</a> has a main() that launches off the training job.

In [81]:
%bash
grep "^def" txtcls1/trainer/model.py

def init(bucket, num_epochs):
def save_vocab(trainfile, txtcolname, outfilename):
def read_dataset(prefix, batch_size=20):
def cnn_model(features, target, mode):
def linear_model(features, target, mode):
def serving_input_fn():
def get_train():
def get_valid():
def experiment_fn(output_dir):


Let's make sure the code works locally on a small dataset for a few epochs.

In [122]:
%bash
echo "bucket=${BUCKET}"
rm -rf outputdir
export PYTHONPATH=${PYTHONPATH}:${PWD}/txtcls1
python -m trainer.task \
   --bucket=${BUCKET} \
   --output_dir=outputdir \
   --job-dir=./tmp --num_epochs=2

bucket=cloud-training-demos-ml
6355 words in gs://cloud-training-demos-ml/txtcls1/train.csv being written to gs://cloud-training-demos-ml/txtcls1/vocab_words
lookup_words=Tensor("word_to_index_Lookup:0", shape=(?, ?), dtype=int64)
words_sliced=Tensor("Slice:0", shape=(?, 20), dtype=int64)
words_embed=Tensor("words/embedding_lookup:0", shape=(?, 20, 10), dtype=float32)


Copying gs://cloud-training-demos-ml/txtcls1/train.csv...
| [1 files][  4.0 MiB/  4.0 MiB]                                                
Operation completed over 1 objects/4.0 MiB.                                      
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1cdac6ce10>, '_model_dir': 'outputdir/', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
Traceback (most recent call last):
  File "/usr/

When I ran it, I got a 30% accuracy in two epochs. Which is essentially a coin toss or whatever you call something with 1/3 odds ...
<p>
Once the code works in standalone mode, you can run it on Cloud ML Engine. You can monitor the job from the GCP console in the Cloud Machine Learning Engine section.

In [99]:
%bash
OUTDIR=gs://${BUCKET}/txtcls1/trained_model
JOBNAME=txtcls_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gsutil cp txtcls1/trainer/*.py $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=$(pwd)/txtcls1/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC --runtime-version=1.2 \
   -- \
   --bucket=${BUCKET} \
   --output_dir=${OUTDIR} \
   --num_epochs=10

gs://cloud-training-demos-ml/txtcls1/trained_model us-central1 txtcls_170620_220206
jobId: txtcls_170620_220206
state: QUEUED


Removing gs://cloud-training-demos-ml/txtcls1/trained_model/#1497904932113739...
Removing gs://cloud-training-demos-ml/txtcls1/trained_model/__init__.py#1497904590096433...
Removing gs://cloud-training-demos-ml/txtcls1/trained_model/export/Servo/1497904940/assets/#1497904944025067...
Removing gs://cloud-training-demos-ml/txtcls1/trained_model/checkpoint#1497904933774605...
Removing gs://cloud-training-demos-ml/txtcls1/trained_model/eval/events.out.tfevents.1497904939.master-d3c4dcd8c0-0-kkkjr#1497904940174974...
Removing gs://cloud-training-demos-ml/txtcls1/trained_model/eval/#1497904922743917...
Removing gs://cloud-training-demos-ml/txtcls1/trained_model/events.out.tfevents.1497904906.master-d3c4dcd8c0-0-kkkjr#1497904934854218...
Removing gs://cloud-training-demos-ml/txtcls1/trained_model/eval/events.out.tfevents.1497904922.master-d3c4dcd8c0-0-kkkjr#1497904923798317...
Removing gs://cloud-training-demos-ml/txtcls1/trained_model/export/Servo/1497904940/assets/vocab_words#14979049446323

Training finished with an accuracy of 61%.  Obviously, this was trained on a really small dataset. But I hope the sample works for you to apply to your *real* data.

<h2> Deploy trained model </h2>
<p>
Deploying the trained model to act as a REST web service is a simple gcloud call.

In [11]:
%bash
gsutil ls gs://${BUCKET}/txtcls1/trained_model/export/Servo/

gs://cloud-training-demos-ml/txtcls1/trained_model/export/Servo/
gs://cloud-training-demos-ml/txtcls1/trained_model/export/Servo/1497904940/


In [12]:
%bash
MODEL_NAME="txtcls"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/txtcls1/trained_model/export/Servo/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION}

Deleting and deploying txtcls v1 from gs://cloud-training-demos-ml/txtcls1/trained_model/export/Servo/1497904940/ ... this will take a few minutes


Creating version (this might take a few minutes)......
................................................................................done.


<h2> Use model to predict </h2>
<p>
Send a JSON request to the endpoint of the service to make it predict which publication the article is more likely to run in. These are actual titles of articles in the New York Times, TechCrunch, and Wired on June 19.

In [18]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1beta1', credentials=credentials,
            discoveryServiceUrl='https://storage.googleapis.com/cloud-ml/discovery/ml_v1beta1_discovery.json')

request_data = {'instances':
  [
      {
        'inputs': 'Supreme Court to Hear Major Case on Partisan Districts'
      },
      {
        'inputs': 'Time Warner will spend $100M on Snapchat original shows and ads'
      },
      {
        'inputs': 'This Dark Matter Theory Could Solve a Celestial Conundrum'
      },
  ]
}

parent = 'projects/%s/models/%s/versions/%s' % (PROJECT, 'txtcls', 'v1')
response = api.projects().predict(body=request_data, name=parent).execute()
print "response={0}".format(response)

response={u'predictions': [{u'source': u'nytimes', u'prob': [0.6047254800796509, 0.24059338867664337, 0.15468111634254456], u'class': 0}, {u'source': u'nytimes', u'prob': [0.6047254800796509, 0.24059338867664337, 0.15468111634254456], u'class': 0}, {u'source': u'nytimes', u'prob': [0.6047254800796509, 0.24059338867664337, 0.15468111634254456], u'class': 0}]}


According to the model, our son would have clocked in at 7.3 lbs and our daughter at 6.8 lbs.
<p>
The weights are off by about 0.5 lbs. Pretty cool!

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License